In [8]:
import argparse
from logging import getLogger
from tqdm import tqdm
import torch
import numpy as np
import pandas as pd
import pickle

from recbole.quick_start import load_data_and_model
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color
from recbole.data import create_dataset
from recbole.utils.case_study import full_sort_topk, full_sort_scores

import logging
from logging import getLogger

import torch
import pickle

from recbole.config import Config
from recbole.data import create_dataset, data_preparation, save_split_dataloaders, load_split_dataloaders
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color

import copy
from recbole.data.interaction import Interaction, cat_interactions
import os
from collections import OrderedDict

pd.set_option('display.max_rows', None)


In [9]:
def get_result(log):
    with os.popen(f'tail -n 1 {log}') as f:
        content = f.read()
    # print(log, repr(content))
    result = eval(content.split('test result: ')[1])
    return result

In [10]:
sasrec_trans_result = get_result('output/xining/0112-exp/up-\[30\,inf\)/down-\[4\,20\]/SASRec.trans.log')
sasrecfeat_trans_result = get_result('output/xining/0112-exp/up-\[30\,inf\)/down-\[4\,20\]/SASRecFeat.trans.log')
fusion_trans_result = get_result('output/xining/0112-exp/up-\[30\,inf\)/down-\[4\,20\]/fusion.trans.log')

In [17]:
user_list = list(sasrec_trans_result.keys())
metrics = list(sasrec_trans_result[user_list[0]].keys())

In [20]:
def get_result_matrix(result):
    matrix = np.zeros((len(user_list), len(metrics)))
    for i, user in enumerate(user_list):
        for j, metric in enumerate(metrics):
            matrix[i, j] = result[user][metric]
    return matrix

In [21]:
sasrec_trans_result_matrix = get_result_matrix(sasrec_trans_result)
sasrecfeat_trans_result_matrix = get_result_matrix(sasrecfeat_trans_result)
fusion_trans_result_matrix = get_result_matrix(fusion_trans_result)

In [26]:
result_matrix = np.concatenate([sasrec_trans_result_matrix, sasrecfeat_trans_result_matrix, fusion_trans_result_matrix], axis=-1)
result_matrix = result_matrix.reshape((len(user_list) * 3, len(metrics)))

In [27]:
result_df = pd.DataFrame(result_matrix, columns=metrics, index=pd.MultiIndex.from_product([user_list, ['SASRec', 'SASRecFeat', 'Fusion']]))

In [28]:
result_df

recall@10  mrr@10  ndcg@10  hit@10  precision@10
3336141157 SASRec            0.0  0.0000   0.0000     0.0           0.0
           SASRecFeat        0.0  0.0000   0.0000     0.0           0.0
           Fusion            0.0  0.0000   0.0000     0.0           0.0
2486704301 SASRec            1.0  0.2500   0.4307     1.0           0.1
           SASRecFeat        1.0  0.5000   0.6309     1.0           0.1
           Fusion            0.0  0.0000   0.0000     0.0           0.0
1811520602 SASRec            1.0  0.2500   0.4307     1.0           0.1
           SASRecFeat        0.0  0.0000   0.0000     0.0           0.0
           Fusion            0.0  0.0000   0.0000     0.0           0.0
3013134143 SASRec            0.0  0.0000   0.0000     0.0           0.0
           SASRecFeat        1.0  0.2000   0.3869     1.0           0.1
           Fusion            0.0  0.0000   0.0000     0.0           0.0
2270387010 SASRec            1.0  0.3333   0.5000     1.0           0.1
           SASRecFeat        0.0  0.0000   0.0000     0.0           0.0
           Fusion            0.0  0.0000   0.0000     0.0           0.0
907781480  SASRec            0.0  0.0000   0.0000     0.0           0.0
           SASRecFeat        0.0  0.0000   0.0000     0.0           0.0
           Fusion            0.0  0.0000   0.0000     0.0           0.0
1823033342 SASRec            0.0  0.0000   0.0000     0.0           0.0
           SASRecFeat        0.0  0.0000   0.0000     0.0           0.0
           Fusion            0.0  0.0000   0.0000     0.0           0.0
3176893828 SASRec            0.0  0.0000   0.0000     0.0           0.0
           SASRecFeat        0.0  0.0000   0.0000     0.0           0.0
           Fusion            0.0  0.0000   0.0000     0.0           0.0
2893893535 SASRec            1.0  0.1429   0.3333     1.0           0.1
           SASRecFeat        1.0  0.2000   0.3869     1.0           0.1
           Fusion            0.0  0.0000   0.0000     0.0           0.0
125138683  SASRec            0.0  0.0000   0.0000     0.0           0.0
           SASRecFeat        0.0  0.0000   0.0000     0.0           0.0
           Fusion            0.0  0.0000   0.0000     0.0           0.0
2975420932 SASRec            0.0  0.0000   0.0000     0.0           0.0
           SASRecFeat        0.0  0.0000   0.0000     0.0           0.0
           Fusion            0.0  0.0000   0.0000     0.0           0.0
208358440  SASRec            0.0  0.0000   0.0000     0.0           0.0
           SASRecFeat        0.0  0.0000   0.0000     0.0           0.0
           Fusion            0.0  0.0000   0.0000     0.0           0.0
801934877  SASRec            1.0  0.1429   0.3333     1.0           0.1
           SASRecFeat        0.0  0.0000   0.0000     0.0           0.0
           Fusion            0.0  0.0000   0.0000     0.0           0.0
206044036  SASRec            1.0  0.2500   0.4307     1.0           0.1
           SASRecFeat        1.0  1.0000   1.0000     1.0           0.1
           Fusion            1.0  1.0000   1.0000     1.0           0.1
1722190258 SASRec            0.0  0.0000   0.0000     0.0           0.0
           SASRecFeat        0.0  0.0000   0.0000     0.0           0.0
           Fusion            0.0  0.0000   0.0000     0.0           0.0
287255803  SASRec            1.0  1.0000   1.0000     1.0           0.1
           SASRecFeat        1.0  0.5000   0.6309     1.0           0.1
           Fusion            0.0  0.0000   0.0000     0.0           0.0
3400861982 SASRec            1.0  1.0000   1.0000     1.0           0.1
           SASRecFeat        1.0  1.0000   1.0000     1.0           0.1
           Fusion            1.0  1.0000   1.0000     1.0           0.1
3210631559 SASRec            0.0  0.0000   0.0000     0.0           0.0
           SASRecFeat        0.0  0.0000   0.0000     0.0           0.0
           Fusion            0.0  0.0000   0.0000     0.0           0.0
3398077491 SASRec            0.0  0.0000   0.0000     0.0      

In [29]:
user = '2486704301'
sasrec_trans_result[user]

OrderedDict([('recall@10', 1.0),
             ('mrr@10', 0.25),
             ('ndcg@10', 0.4307),
             ('hit@10', 1.0),
             ('precision@10', 0.1)])

In [30]:
sasrecfeat_trans_result[user]

OrderedDict([('recall@10', 1.0),
             ('mrr@10', 0.5),
             ('ndcg@10', 0.6309),
             ('hit@10', 1.0),
             ('precision@10', 0.1)])

In [31]:
fusion_trans_result[user]

OrderedDict([('recall@10', 0.0),
             ('mrr@10', 0.0),
             ('ndcg@10', 0.0),
             ('hit@10', 0.0),
             ('precision@10', 0.0)])